Relevant directions from https://adventofcode.com/2023/day/2 part two -

As you continue your walk, the Elf poses a second question: in each game you played, what is the fewest number of cubes of each color that could have been in the bag to make the game possible?

Again consider the example games from earlier:

```
Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
```

- In game 1, the game could have been played with as few as 4 red, 2 green, and 6 blue cubes. If any color had even one fewer cube, the game would have been impossible.
- Game 2 could have been played with a minimum of 1 red, 3 green, and 4 blue cubes.
- Game 3 must have been played with at least 20 red, 13 green, and 6 blue cubes.
- Game 4 required at least 14 red, 3 green, and 15 blue cubes.
- Game 5 needed no fewer than 6 red, 3 green, and 2 blue cubes in the bag.

The power of a set of cubes is equal to the numbers of red, green, and blue cubes multiplied together. The power of the minimum set of cubes in game 1 is 48. In games 2-5 it was 12, 1560, 630, and 36, respectively. Adding up these five powers produces the sum **2286**.

For each game, find the minimum set of cubes that must have been present. What is the sum of the power of these sets?

In [2]:
let text = r#"
Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green
Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue
Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red
Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red
Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green
"#;
let lines: Vec<&str> = text.trim().lines().collect();
lines

["Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green", "Game 2: 1 blue, 2 green; 3 green, 4 blue, 1 red; 1 green, 1 blue", "Game 3: 8 green, 6 blue, 20 red; 5 blue, 4 red, 13 green; 5 green, 1 red", "Game 4: 1 green, 3 red, 6 blue; 3 green, 6 red; 3 green, 15 blue, 14 red", "Game 5: 6 red, 1 blue, 3 green; 2 blue, 1 red, 2 green"]

In [3]:
#[derive(Debug)]
struct Bag {
    red: i32,
    green: i32,
    blue: i32,
}

impl Bag {
    fn new(red: i32, green: i32, blue: i32) -> Bag {
        Bag { red, green, blue }
    }

    fn from_string(s: &str) -> Result<Bag, &'static str> {
        let mut red = 0;
        let mut green = 0;
        let mut blue = 0;

        for part in s.split(',') {
            let parts: Vec<&str> = part.trim().split_whitespace().collect();
            if parts.len() != 2 {
                return Err("Invalid input format");
            }

            let count = parts[0].parse::<i32>().map_err(|_| "Invalid count")?;
            match parts[1] {
                "red" => red = count,
                "green" => green = count,
                "blue" => blue = count,
                _ => return Err("Unknown color"),
            }
        }

        Ok(Bag::new(red, green, blue))
    }

    fn power(&self) -> i32 {
        self.red * self.green * self.blue
    }
}

let example = Bag::from_string("8 green, 6 blue, 20 red").unwrap();
example

Bag { red: 20, green: 8, blue: 6 }

In [4]:
#[derive(Debug)]
struct Game {
    id: i32,
    bags: Vec<Bag>,
}

impl Game {
    fn from_string(s: &str) -> Result<Game, &'static str> {
        let parts: Vec<&str> = s.split(':').collect();
        if parts.len() != 2 {
            return Err("Invalid input format");
        }

        let id = parts[0].split_whitespace().last().ok_or("Invalid ID")?
            .parse::<i32>().map_err(|_| "ID is not a number")?;

        let bag_parts = parts[1].split(';');
        let mut bags = Vec::new();

        for part in bag_parts {
            match Bag::from_string(part) {
                Ok(bag) => bags.push(bag),
                Err(e) => return Err(e),
            }
        }

        Ok(Game { id, bags })
    }

    fn max_values(&self) -> Bag {
        let mut max_red = 0;
        let mut max_green = 0;
        let mut max_blue = 0;

        for bag in &self.bags {
            max_red = max_red.max(bag.red);
            max_green = max_green.max(bag.green);
            max_blue = max_blue.max(bag.blue);
        }

        Bag::new(max_red, max_green, max_blue)
    }
}

let game = Game::from_string("Game 1: 3 blue, 4 red; 1 red, 2 green, 6 blue; 2 green\n").unwrap();
game

Game { id: 1, bags: [Bag { red: 4, green: 0, blue: 3 }, Bag { red: 1, green: 2, blue: 6 }, Bag { red: 0, green: 2, blue: 0 }] }

In [5]:
game.max_values()

Bag { red: 4, green: 2, blue: 6 }

In [6]:
game.max_values().power()

48

In [7]:
let mut games = Vec::new();
for line in &lines {
    let game = Game::from_string(&line).unwrap();
    games.push(game);
};
games

[Game { id: 1, bags: [Bag { red: 4, green: 0, blue: 3 }, Bag { red: 1, green: 2, blue: 6 }, Bag { red: 0, green: 2, blue: 0 }] }, Game { id: 2, bags: [Bag { red: 0, green: 2, blue: 1 }, Bag { red: 1, green: 3, blue: 4 }, Bag { red: 0, green: 1, blue: 1 }] }, Game { id: 3, bags: [Bag { red: 20, green: 8, blue: 6 }, Bag { red: 4, green: 13, blue: 5 }, Bag { red: 1, green: 5, blue: 0 }] }, Game { id: 4, bags: [Bag { red: 3, green: 1, blue: 6 }, Bag { red: 6, green: 3, blue: 0 }, Bag { red: 14, green: 3, blue: 15 }] }, Game { id: 5, bags: [Bag { red: 6, green: 3, blue: 1 }, Bag { red: 1, green: 2, blue: 2 }] }]

In [8]:
for game in &games {
    println!("ID {}. Power: {}", &game.id, game.max_values().power());
}

ID 1. Power: 48
ID 2. Power: 12
ID 3. Power: 1560
ID 4. Power: 630
ID 5. Power: 36


()

In [9]:
let mut power_sum = 0;
for game in &games {
    power_sum += game.max_values().power();
}
power_sum

2286